In [ ]:
%pylab inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if True:
    from PIL import Image    
    im = Image.open("./resources/testbed_0.2.png")
    #im = Image.open("./reference_new2.bmp")
    img = np.array(im)
    print( img.shape) # Ydim, Xdim
else:
    values=np.fromfile('build/spots.bin', dtype='uint8')
    img=values.reshape([int(values.shape[0]**0.5)]*2)

In [ ]:
img.tofile('build/spots_matlab.bin','') # No sep to force binary

In [ ]:
8192*8192* 4

In [ ]:
1024*16

In [ ]:
16384 * 16384

In [ ]:
#cx-boxes_x

In [ ]:
#mg[cx-perside//2,]

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(img, cmap='bone')
#plt.imsave('spots.png', img)

#cx,cy=[1038,1038]
cx,cy=[500,500]
perside=40

def box1(cx,cy):
    plt.plot( [cx-perside//2,cx+perside//2], [cy-perside//2]*2,'r-', lw=2)
    plt.plot( [cx-perside//2,cx+perside//2], [cy+perside//2]*2,'r-', lw=2)
    plt.plot( [cx-perside//2]*2, [cy-perside//2,cy+perside//2], 'r-', lw=2)
    plt.plot( [cx+perside//2]*2, [cy-perside//2,cy+perside//2], 'r-', lw=2)

box1(cx,cy)

In [ ]:
437 * 4 * 2

In [ ]:
img.shape

In [ ]:
670/2

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x):
    demo_boxes(x)
    return x

interact(f, x=widgets.IntSlider(min=300, max=1000, step=1, value=500));

In [ ]:
def demo_boxes(pupil_radius_pixel,cx=518,cy=488, limit=12):

    # Generate initial searchboxes.
    # Inspired by the MATLAB code, but converted to matrix ops instead of loops
    #pupil_radius_pixel = 2000 #np.max(img.shape)//2
    #box_size_pixel = 100/6.4
    #pupil_radius_pixel = np.max(img.shape)//2
    #print( "WHAT", pupil_radius_pixel)

    img_max = np.max(img.shape)//2
    
    #pupil_radius_pixel = 620

    cx=518
    cy=488

    #box_size_pixel = 40
    BOX_UM=328
    CCD_PIXEL=5.5*2
    #CCD_PIXEL=6.4

    print( "pupil u", pupil_radius_pixel * CCD_PIXEL)
    box_size_pixel=BOX_UM/CCD_PIXEL
    print( "box", box_size_pixel)

    # How many "boxwidths" (radius) in the pupil
    ri_ratio = pupil_radius_pixel / box_size_pixel

    print( "ri rat", ri_ratio)
    
    # The max number of boxes possible + or -
    max_boxes = np.ceil( pupil_radius_pixel/ box_size_pixel )

    # All possible bilinear box
    boxes_x = np.arange(-max_boxes,max_boxes+1) # +1 to include +max_boxes number
    boxes_y = np.arange(-max_boxes,max_boxes+1)

    # Determine outer edge of each box using corners away from the center:
    # 0.5*sign: positive adds 0.5, negative substracts 0.5
    #
    # Filter valid to only those where the radius is within ri_ratio
    XX,YY = np.meshgrid(boxes_x, boxes_y )
    RR = np.sqrt( (XX+0.5*np.sign(XX))**2 + (YY+0.5*np.sign(YY))**2 )
    valid_boxes = np.where(RR<limit)
    max_dist_boxwidths = np.max(RR[RR<limit])

    # Normalize to range -1 .. 1 (vs. pupil size)
    valid_x_norm=XX[valid_boxes]/ri_ratio
    valid_y_norm=YY[valid_boxes]/ri_ratio

    num_boxes=valid_x_norm.shape[0]
    print( num_boxes, np.min(valid_x_norm), np.min(valid_y_norm))
    print( num_boxes, np.max(valid_x_norm), np.max(valid_y_norm))
    
    box_zero = np.where(valid_x_norm**2+valid_y_norm**2==0)[0] # Locate the central box

    valid_x = valid_x_norm * 500  + cx
    valid_y = valid_y_norm * 500  + cy
    plt.figure(figsize=(8,8))

    plt.imshow(img, cmap='bone')

    plt.scatter( valid_x, valid_y, s=2)
    plt.scatter( valid_x[box_zero], valid_y[box_zero], s=10, color='red')

    circle1 = plt.Circle((cx, cy), pupil_radius_pixel, color='r', fill=False)
    plt.gca().add_patch(circle1)
    plt.axis('equal')

In [ ]:
lefts = valid_x_norm-0.5/ri_ratio
rights = valid_x_norm+0.5/ri_ratio
ups = -( valid_y_norm+0.5/ri_ratio )
downs = -( valid_y_norm-0.5/ri_ratio )

In [ ]:
ri_ratio

In [ ]:
import zernike_integrals
from numpy.linalg import svd,lstsq

In [ ]:
lenslet_dx,lenslet_dy=zernike_integrals.zernike_integral_average_from_corners(
    lefts,rights,ups,downs,6.4)
#lenslet_dx /= 6400/1000/2 # pupil
#lenslet_dy /= 6400/1000/2 # puil

In [ ]:
start_zc=1 # Zero-based, skip first term
num_zcs=68

lenslet_dx = lenslet_dx[start_zc:num_zcs,:].T
lenslet_dy = lenslet_dy[start_zc:num_zcs,:].T

In [ ]:
lenslet_dx

In [ ]:
zpoly = np.hstack( (lenslet_dx.T, lenslet_dy.T ) ).T
[uu,ss,vv] = svd(zpoly,False)
ss_full = eye(ss.shape[0])*ss

In [ ]:
import scipy.io
mat_solution=scipy.io.loadmat('../miniwave/matlab_centroid_ui/Zernike fit to spot centroids/svd_vars.mat')

In [ ]:
uu.shape,ss.shape,ss_full.shape,vv.shape

In [ ]:
leftside = numpy.linalg.lstsq(ss_full, vv, rcond=0)[0].T # Python equiv to MATLAB's vv/ss (solving system of eqns)
zterms = np.matmul( leftside, uu.T)
coeff=np.matmul(zterms,mat_solution['slope'])

#zernikeCVS=np.zeros(num_zcs)
#zernikeCVS[start_zc:num_zcs] = coeff

In [ ]:
plt.plot( coeff[0:10], 'o' )

In [ ]:
plt.plot( mat_solution['coeff'][0:10], 'o' )

In [ ]:
what = lstsq( zterms, np.eye(coeff.shape[0])*coeff,rcond=0 )[0].T

In [ ]:
inver=np.matmul( zterms.T, coeff )

In [ ]:
inver=np.linalg.pinv(zterms)
print( inver.shape, zterms.shape )

In [ ]:
shape(zterms), shape(inver)

In [ ]:
1024 * 1024

In [ ]:
num_boxes * 2, 

In [ ]:
coeff *= 0;
coeff[2:4]=2.5
plt.plot(coeff)

In [ ]:
plt.plot( np.matmul(inver, coeff) )

In [ ]:
plt.plot( mat_solution['slope'] )

In [ ]:
coeff

In [ ]:
x=np.array([1,2,3]); y=np.array([4,5,6])
wow=np.vstack((x,y)).T
wow.shape


In [ ]:
# Copied from MATLAB code
CVS_to_OSA_map = np.array([3,2, 5,4,6, 9,7,8,10, 15,13,11,12,14,
                           21,19,17,16,18,20,
                           27,25,23,22,24,26,28, 35,33,31,29,30,32,34,36,
                           45,43,41,39,37,38,40,42,44,
                           55,53,51,49,47,46,48,50,52,54,
                           65,63,61,59,57,56,58,60,62,64,66,67,68,69,70])

zernike_OSA=coeff[CVS_to_OSA_map[start_zc-1:num_zcs-1]-start_zc-1 ]

In [ ]:
coeff[1]

In [ ]:
zernike_OSA[0]

In [ ]:
CVS_to_OSA_map[start_zc-1:num_zcs-1]-start_zc-1

In [ ]:
idxs=CVS_to_OSA_map[start_zc-1:num_zcs-1]-start_zc-1

In [ ]:
np.array( [np.where(CVS_to_OSA_map[:-4]-2==n)[0][0] for n in np.arange(10) ] )

In [ ]:
#list(zip(np.arange(len(CVS_to_OSA_map)), CVS_to_OSA_map) )

In [ ]:
np.max( zernike_OSA - mat_solution['zernikeOSA'][0:67] )

In [ ]:
np.max(coeff - mat_solution['coeff'] )

In [ ]:
np.max(zterms - mat_solution['zterms'] )

In [ ]:
vv/(eye(ss.shape[0])*ss)*uu.T

In [ ]:
zt1=np.load('zterms.npy')
slope=np.load('slope.npy')

In [ ]:
np.max( zt1-zterms )

In [ ]:
np.max(mat_solution['slope']-slope)

In [ ]:
np.shape(mat_solution['slope'][0])

In [ ]:
plot(slope-1.5);
plot(mat_solution['slope'] )

In [ ]:
mat_solution['slope']-slope

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(img, cmap='bone')
box1(valid_x[box_zero], valid_y[box_zero])
box1(valid_x[0], valid_y[0])
box1(valid_x[8], valid_y[8])
box1(valid_x[-1], valid_y[-1])

In [ ]:
plt.imshow(img[60:70,340:350])

In [ ]:
layout=extract_memory.get_header_format('memory_layout.h')    

In [ ]:
defs,size

In [ ]:
# https://stackoverflow.com/questions/63553692/how-to-use-memory-mapped-file-in-python-linux


In [ ]:
import extract_memory
size,fields,defs=extract_memory.get_header_format('layout_boxes.h')

In [ ]:
import extract_memory
size,fields,defs=extract_memory.get_header_format('layout_boxes.h')

import struct

import os
import mmap
NUM_BOXES=defs['MAX_BOXES']
fd = os.open('/dev/shm/NW_BUFFER_BOXES',os.O_RDWR)
shmem_boxes=mmap.mmap(fd,size)


class ByteStream(bytearray):
    def append(self, v, fmt='>B'):
        self.extend(struct.pack(fmt, v))

buf = ByteStream()
for item in valid_x:
    buf.append(item, 'f')
shmem_boxes.seek(fields['reference_x']['bytenum_current'])
shmem_boxes.write(buf)
shmem_boxes.flush()
               
buf = ByteStream()
for item in valid_y:
    buf.append(item, 'f')
shmem_boxes.seek(fields['reference_y']['bytenum_current'])
shmem_boxes.write(buf)
shmem_boxes.flush()

# Write header last, so the engine knows when we are ready
buf = ByteStream()
buf.append(1)
buf.append(2)
buf.append(num_boxes, 'H')
buf.append(box_size_pixel, 'd')
buf.append(pupil_radius_pixel, 'd')
shmem_boxes.seek(0)
shmem_boxes.write(buf)
shmem_boxes.flush()

os.close(fd)

In [ ]:
cd ..

In [ ]:
shmem_boxes.seek(fields['centroid_x']['bytenum_current'])
buf=shmem_boxes.read(num_boxes*4)
centroids_x=struct.unpack_from(''.join((['f']*num_boxes)), buf)

shmem_boxes.seek(fields['centroid_y']['bytenum_current'])
buf=shmem_boxes.read(num_boxes*4)
centroids_y=struct.unpack_from(''.join((['f']*num_boxes)), buf)


In [ ]:
plt.plot( centroids_x, centroids_y, '.')
plt.axis('equal')

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img, cmap='bone')
plt.plot( centroids_x, centroids_y, 'r.', ms=2)

plt.plot( ref_x, ref_y, 'b.', ms=2)

In [ ]:
#buf


In [ ]:
0xb5 * 256 + 0x1

In [ ]:
plt.figure(figsize=(12,4))
plt.plot( img.mean(1) )

In [ ]:
1038,1038